<a href="https://colab.research.google.com/github/zachroyer/Music-Symbol-Classification/blob/main/HW7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
#@title Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

# Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


# Part 1: Load/Analyze Data

##What is your dataset? Please describe your dataset and where you got it

My dataset of choice is titled "Music Notes Dataset". I obtained it on Kaggle uploaded by a user 'kishanj'.

## Description of Music Notes Dataset
This dataset contains a large set of one category of notes one might encounter while examining sheet music.

One image can correspond to the following labels:
*  Whole
*  Half
*  Quarter
*  Eighth
*  Sixteenth

Each note can be broken down in to three components (or a lack of one or more):
1.   Note Head
2.   Stem
3.   Flag

INSERT IMAGE

The change of these components is what determines how *long* a note is held for when playing it. The placement of the note determines *which* note to play on your desired instrument. This dataset omits that implicit feature by isolating the images to the features of a note, not its placement.

The Note Head is the circular portion of the note.
The Stem is the thin line that grows from the Note Head.
The Flag is the perpindicular tail on the end of a Stem.

# Self-Collected Test Data
In addition to the dataset from Kaggle, I also collected a small sample of test data myself. The data came from a digital scan of a physical copy of Minuet in D Minor (J.S. Bach) from the Schirmer's Library of Musical Classics Vol. 2066. This is a piece that I have personally studied on the piano.

With that digital scan,


##Load your dataset / Implement your dataloader

In [61]:
# Installing dataset directly from Kaggle
!pip install kagglehub

import kagglehub
path = kagglehub.dataset_download("kishanj/music-notes-datasets")
print("Downloaded to: ", path)

100%|██████████| 12.0M/12.0M [00:01<00:00, 12.1MB/s]

Extracting files...


Downloaded to:  /root/.cache/kagglehub/datasets/kishanj/music-notes-datasets/versions/4


In [69]:
# Defining a Transform
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels = 1), # Guaranteeing only one color channel for efficiency
    transforms.Resize((64,64)), # Resizing image
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5)), # Normalizing pixel values to range from (0,1) and (-1,1) for stability
])

data = torchvision.datasets.ImageFolder(root=path, transform=transform) # Easily extract class information for data from directory structure using ImageFolder
print(len(data))

5000


In [77]:
# Splitting Data into random train, validation and test splits
train_split, val_split, test_split = torch.utils.data.random_split(data, [int(len(data)*0.8), int(len(data)*0.1), int(len(data)*0.1)])

# Wrapping random data in DataLoader for PyTorch
train_loader = DataLoader(train_split)
validation_loader = DataLoader(val_split)
test_loader = DataLoader(test_split)

##Dataset Analyis Statistics

In [78]:
# TODO - code for collecting statistics from your dataset / plots and analysis
train_iter = iter(train_loader)
print(f"Number of data points (images) in each split (train/val/test): {len(train_loader)}/{len(validation_loader)}/{len(test_loader)}")
print(f"")

A description of the dataset
● Format of the inputs and outputs
● Number of examples in the train/val/test splits
● A visualization of at least 5 examples from your dataset
Summary statistics (e.g. for language datasets, you might include things such as the average lengths
of the inputs or the number of unique words).
● If possible, any visualizations of the dataset statistics.

Number of data points (images) in each split (train/val/test): 4000/500/500


# Analysis


## Format


# Part 2: Literature Review

TODO - Follow instructions in the assignment for your literature review


https://ieeexplore.ieee.org/document/8270207 (Accessible via University of Utah login)

# Part 3: Implement networks in PyTorch

Note of warning here: Depending on how easily you can allocate GPU resources, you may want to make your network much shallower so that you can train it more easily
Aim to have one gradient update take no more than a few seconds
May also want to reduce the number of training steps if training is too slow

In [ ]:
# TODO - implement your PyTorch Module


In [ ]:
# TODO - implement your training loop

In [ ]:
# TODO - plot train and validation accuracy of your model during training

Hint: look at earlier homeworks / resuse code from those to help you here

# Part 4: Run Hyperparameter Experiments

In [ ]:
# TODO - perform hyperparameter grid searches and plot accuracies

In [ ]:
# TODO - perform other experiments and plot accuracies

In [ ]:
# TODO - calculate the final test accuracy

TODO - Explain your experiments above. See assignment doc for more details.